In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:

#@title We Load Our Data Pertaining to the Facilitators, Activites, Timeslots, and  Rooms & Capacities

def load_facilitators(filename):
    with open(filename, 'r') as file:
        facilitators = file.read().strip().split(", ")
    return facilitators

# Step 2: Load Time Slots
def load_time_slots(filename):
    with open(filename, 'r') as file:
        time_slots = file.read().strip().split("\n")
    return time_slots

# Step 3: Load Rooms and Capacities
def load_rooms(filename):
    rooms = {}
    with open(filename, 'r') as file:
        lines = file.readlines()
        for line in lines:
            room_data = line.strip().split(" ")
            room_name = room_data[0]
            capacity = int(room_data[1])
            rooms[room_name] = capacity
    return rooms


def load_activities(filename):
    activities = {}
    with open(filename, 'r') as file:
        content = file.read().strip()

        # Split content by blocks of activities based on empty lines
        activities_data = content.split("\n\n")

        for activity_data in activities_data:
            lines = activity_data.split("\n")

            # Get activity names, checking if there are multiple names
            activity_names = [name.strip() for name in lines[0].split(",")]

            # Extract expected enrollment
            expected_enrollment = int(lines[1].split(": ")[1])

            # Extract preferred facilitators
            preferred_facilitators = lines[2].split(": ")[1].split(", ")

            # Extract other facilitators if present
            if len(lines) > 3 and lines[3].startswith("Other facilitators"):
                other_facilitators = lines[3].split(": ")[1].split(", ")
            else:
                other_facilitators = []

            # Save data for each activity name
            for activity_name in activity_names:
                activities[activity_name] = {
                    "expected_enrollment": expected_enrollment,
                    "preferred_facilitators": preferred_facilitators,
                    "other_facilitators": other_facilitators
                }
    return activities




facilitators = load_facilitators('/content/drive/MyDrive/Genetic Algorithms Assignment Files/FacilitatorsList461Assign2.txt')
time_slots = load_time_slots('/content/drive/MyDrive/Genetic Algorithms Assignment Files/TimeList461Assign2.txt')
rooms = load_rooms('/content/drive/MyDrive/Genetic Algorithms Assignment Files/RoomList461Assign2.txt')
activities = load_activities('/content/drive/MyDrive/Genetic Algorithms Assignment Files/RoomDataList461Assign2.txt')

# Print loaded data for verification
print("Facilitators:", facilitators)
print("Time Slots:", time_slots)
print("Rooms:", rooms)
print("Activities:", activities)

Mounted at /content/drive
Facilitators: ['Lock', 'Glen', 'Banks', 'Richards', 'Shaw', 'Singer', 'Uther', 'Tyler', 'Numen', 'Zeldin']
Time Slots: ['10AM', '11AM', '12PM', '1PM', '2PM', '3PM']
Rooms: {'Slater-003': 45, 'Roman-216': 30, 'Loft-206': 75, 'Roman-201': 50, 'Loft-310': 108, 'Beach-201': 60, 'Beach-301': 75, 'Logos-325': 450, 'Frank-119': 60}
Activities: {'SLA100A': {'expected_enrollment': 50, 'preferred_facilitators': ['Glen', 'Lock', 'Banks', 'Zeldin'], 'other_facilitators': ['Numen', 'Richards']}, 'SLA100B:': {'expected_enrollment': 50, 'preferred_facilitators': ['Glen', 'Lock', 'Banks', 'Zeldin'], 'other_facilitators': ['Numen', 'Richards']}, 'SLA191A': {'expected_enrollment': 50, 'preferred_facilitators': ['Glen', 'Lock', 'Banks', 'Zeldin '], 'other_facilitators': ['Numen', 'Richards']}, 'SLA191B:': {'expected_enrollment': 50, 'preferred_facilitators': ['Glen', 'Lock', 'Banks', 'Zeldin '], 'other_facilitators': ['Numen', 'Richards']}, 'SLA201:': {'expected_enrollment': 50,

In [3]:
#@title Setting Up Encoding Mapping for our Values
#We Encode Our Values to Binary For Computer Friendly Operations

import math

def to_binary(n, bits):
    """Convert an integer to a binary string with a fixed number of bits."""
    return format(n, f'0{bits}b')

def create_sequential_mapping(items):
    """Create a sequential mapping for a list of items to binary strings."""
    num_bits = math.ceil(math.log2(len(items)))  # Determine the bits needed
    return {item: to_binary(i, num_bits) for i, item in enumerate(items)}

# Load data (using the functions provided above)
facilitators = load_facilitators('/content/drive/MyDrive/Genetic Algorithms Assignment Files/FacilitatorsList461Assign2.txt')
time_slots = load_time_slots('/content/drive/MyDrive/Genetic Algorithms Assignment Files/TimeList461Assign2.txt')
rooms = load_rooms('/content/drive/MyDrive/Genetic Algorithms Assignment Files/RoomList461Assign2.txt')
activities = load_activities('/content/drive/MyDrive/Genetic Algorithms Assignment Files/RoomDataList461Assign2.txt')

# 1. Facilitator Mapping
facilitator_mapping = create_sequential_mapping(facilitators)
print("Facilitator Mapping:", facilitator_mapping)

# 2. Time Slot Mapping
time_slot_mapping = create_sequential_mapping(time_slots)
print("Time Slot Mapping:", time_slot_mapping)

# 3. Room Mapping
room_mapping = create_sequential_mapping(list(rooms.keys()))
print("Room Mapping:", room_mapping)

# 4. Expected Enrollment Mapping
# Extract unique expected enrollment values from activities
unique_enrollments = sorted({activity['expected_enrollment'] for activity in activities.values()})
enrollment_mapping = create_sequential_mapping(unique_enrollments)
print("Enrollment Mapping:", enrollment_mapping)

Facilitator Mapping: {'Lock': '0000', 'Glen': '0001', 'Banks': '0010', 'Richards': '0011', 'Shaw': '0100', 'Singer': '0101', 'Uther': '0110', 'Tyler': '0111', 'Numen': '1000', 'Zeldin': '1001'}
Time Slot Mapping: {'10AM': '000', '11AM': '001', '12PM': '010', '1PM': '011', '2PM': '100', '3PM': '101'}
Room Mapping: {'Slater-003': '0000', 'Roman-216': '0001', 'Loft-206': '0010', 'Roman-201': '0011', 'Loft-310': '0100', 'Beach-201': '0101', 'Beach-301': '0110', 'Logos-325': '0111', 'Frank-119': '1000'}
Enrollment Mapping: {20: '000', 25: '001', 50: '010', 60: '011', 100: '100'}


In [30]:
#@title Encoding a Schedule

import random

# Step 1: Determine bit lengths for each attribute
room_bits = math.ceil(math.log2(len(room_mapping)))
time_slot_bits = math.ceil(math.log2(len(time_slot_mapping)))
facilitator_bits = math.ceil(math.log2(len(facilitator_mapping)))

# Step 2: Define function to encode a chromosome
def encode_chromosome(activities, room_mapping, time_slot_mapping, facilitator_mapping):
    chromosome = ""

    for activity in activities:
        # Randomly select room, time slot, and facilitator
        room = random.choice(list(room_mapping.values()))
        time_slot = random.choice(list(time_slot_mapping.values()))
        facilitator = random.choice(list(facilitator_mapping.values()))

        # Concatenate binary representations of room, time slot, and facilitator
        assignment = room + time_slot + facilitator
        chromosome += assignment  # Append to the chromosome for this activity

    return chromosome

# Example: Encode a random chromosome
chromosome = encode_chromosome(activities, room_mapping, time_slot_mapping, facilitator_mapping)
print("Encoded Chromosome:", chromosome)


Encoded Chromosome: 0111000010010000010000000001101001000001011101110001000010010000000010011001000000010011000101001100011011011101101000111


In [62]:
#@title Decoding a Schedule

def decode_chromosome(chromosome, activities, room_mapping, time_slot_mapping, facilitator_mapping):
    room_reverse_mapping = {v: k for k, v in room_mapping.items()}
    time_slot_reverse_mapping = {v: k for k, v in time_slot_mapping.items()}
    facilitator_reverse_mapping = {v: k for k, v in facilitator_mapping.items()}

    room_bits = math.ceil(math.log2(len(room_mapping)))
    time_slot_bits = math.ceil(math.log2(len(time_slot_mapping)))
    facilitator_bits = math.ceil(math.log2(len(facilitator_mapping)))
    segment_length = room_bits + time_slot_bits + facilitator_bits

    decoded_schedule = {}
    index = 0

    for activity_name in activities:
        room_binary = chromosome[index:index + room_bits]
        time_slot_binary = chromosome[index + room_bits:index + room_bits + time_slot_bits]
        facilitator_binary = chromosome[index + room_bits + time_slot_bits:index + segment_length]
        index += segment_length

        # Safely decode each segment with default values if out of bounds
        room = room_reverse_mapping.get(room_binary, "DefaultRoom")
        time_slot = time_slot_reverse_mapping.get(time_slot_binary, "DefaultTime")
        facilitator = facilitator_reverse_mapping.get(facilitator_binary, "DefaultFacilitator")

        # Debug print if default values are encountered
        if room == "DefaultRoom" or time_slot == "DefaultTime" or facilitator == "DefaultFacilitator":
            print(f"Warning: Default value encountered during decoding - Room: {room}, Time: {time_slot}, Facilitator: {facilitator}")

        # Store decoded assignment in the schedule dictionary
        decoded_schedule[activity_name] = {
            "room": room,
            "time": time_slot,
            "facilitator": facilitator
        }

    return decoded_schedule

# Example: Decode a chromosome
decoded_schedule = decode_chromosome(chromosome, activities, room_mapping, time_slot_mapping, facilitator_mapping)
print("Decoded Schedule:", decoded_schedule)


Decoded Schedule: {'SLA100A': {'room': 'Roman-216', 'time': '2PM', 'facilitator': 'Lock'}, 'SLA100B:': {'room': 'Beach-301', 'time': '3PM', 'facilitator': 'Lock'}, 'SLA191A': {'room': 'Roman-216', 'time': '2PM', 'facilitator': 'Numen'}, 'SLA191B:': {'room': 'Roman-201', 'time': '10AM', 'facilitator': 'Uther'}, 'SLA201:': {'room': 'Beach-201', 'time': '11AM', 'facilitator': 'Lock'}, 'SLA291:': {'room': 'Beach-301', 'time': '11AM', 'facilitator': 'Tyler'}, 'SLA303:': {'room': 'Loft-310', 'time': '11AM', 'facilitator': 'Zeldin'}, 'SLA304:': {'room': 'Roman-216', 'time': '10AM', 'facilitator': 'Richards'}, 'SLA394:': {'room': 'Beach-201', 'time': '2PM', 'facilitator': 'Singer'}, 'SLA449:': {'room': 'Roman-216', 'time': '1PM', 'facilitator': 'Glen'}, 'SLA451:': {'room': 'Loft-310', 'time': '11AM', 'facilitator': 'Glen'}}


In [32]:
def generate_initial_population(pop_size, activities, room_mapping, time_slot_mapping, facilitator_mapping):
    """Generates an initial population of chromosomes representing schedules."""
    population = []
    for _ in range(pop_size):
        chromosome = encode_chromosome(activities, room_mapping, time_slot_mapping, facilitator_mapping)
        population.append(chromosome)
    return population

In [141]:
#@title Fitness Function
#Note: Had to create a map for our time values to make AM/PM calculations work

def convert_time_to_24_hour(time_str):
    """Convert time string (e.g., '10AM', '1PM') to a 24-hour integer."""
    time_map = {
        "10AM": 10,
        "11AM": 11,
        "12PM": 12,
        "1PM": 13,
        "2PM": 14,
        "3PM": 15
    }
    return time_map[time_str]

def fitness_function(schedule, activities, rooms):
    fitness = 0

    # Track facilitators and time slots for load and conflict checks
    facilitator_schedule = {}
    sla101_times = {}
    sla191_times = {}

    for activity_name, assignment in schedule.items():
        # Start fitness at 0 for each activity
        activity_fitness = 0
        activity = activities[activity_name]
        room = assignment['room']
        time_slot = convert_time_to_24_hour(assignment['time'])
        facilitator = assignment['facilitator']

        # Track time slot for SLA 101 and SLA 191 sections
        if activity_name in ["SLA101A", "SLA101B"]:
            sla101_times[activity_name] = (time_slot, room)
        elif activity_name in ["SLA191A", "SLA191B"]:
            sla191_times[activity_name] = (time_slot, room)

        # Room Capacity Constraints
        expected_enrollment = activity['expected_enrollment']
        room_capacity = rooms[room]

        if room_capacity < expected_enrollment:
            activity_fitness -= 0.5  # Room too small
        elif room_capacity > 3 * expected_enrollment:
            activity_fitness -= 0.2  # Room slightly too large
        elif room_capacity > 6 * expected_enrollment:
            activity_fitness -= 0.4  # Room excessively large
        else:
            activity_fitness += 0.3  # Room appropriately sized

        # Facilitator Preference Constraints
        if facilitator in activity['preferred_facilitators']:
            activity_fitness += 0.5
        elif facilitator in activity['other_facilitators']:
            activity_fitness += 0.2
        else:
            activity_fitness -= 0.1

        # Schedule Conflicts (Same room and time)
        for other_activity, other_assignment in schedule.items():
            if other_activity == activity_name:
                continue

            other_room = other_assignment['room']
            other_time = convert_time_to_24_hour(other_assignment['time'])
            other_facilitator = other_assignment['facilitator']

            # Room Conflict
            if room == other_room and time_slot == other_time:
                activity_fitness -= 0.5  # Penalty for room conflict

            # Facilitator Conflict
            if facilitator == other_facilitator and time_slot == other_time:
                activity_fitness -= 0.2  # Penalty for facilitator conflict

        # Facilitator Load Constraints
        if facilitator not in facilitator_schedule:
            facilitator_schedule[facilitator] = []
        facilitator_schedule[facilitator].append(time_slot)

        same_time_assignments = sum(1 for a in facilitator_schedule[facilitator] if a == time_slot)
        total_assignments = len(facilitator_schedule[facilitator])

        if same_time_assignments == 1:
            activity_fitness += 0.2  # Facilitator has only one activity in this slot
        elif same_time_assignments > 1:
            activity_fitness -= 0.2  # Facilitator has multiple activities in this slot

        if total_assignments > 4:
            activity_fitness -= 0.5
        elif 1 <= total_assignments <= 2 and facilitator != 'Tyler':
            activity_fitness -= 0.4  # Penalty for overseeing only 1 or 2 activities, except for Dr. Tyler

        # Add activity fitness to overall fitness
        fitness += activity_fitness

    # SLA 101 and SLA 191 Special Rules
    if "SLA101A" in sla101_times and "SLA100B" in sla101_times:
        time_diff = abs(sla101_times["SLA100A"][0] - sla101_times["SLA101B"][0])
        if time_diff > 4:
            fitness += 0.5
        elif time_diff == 0:
            fitness -= 0.5

    if "SLA191A" in sla191_times and "SLA191B" in sla191_times:
        time_diff = abs(sla191_times["SLA191A"][0] - sla191_times["SLA191B"][0])
        if time_diff > 4:
            fitness += 0.5
        elif time_diff == 0:
            fitness -= 0.5

    for sla101_section, (time1, room1) in sla101_times.items():
        for sla191_section, (time2, room2) in sla191_times.items():
            time_gap = abs(time1 - time2)
            if time_gap == 1:
                fitness += 0.5
                if (room1 in ["Roman", "Beach"] and room2 not in ["Roman", "Beach"]) or \
                   (room2 in ["Roman", "Beach"] and room1 not in ["Roman", "Beach"]):
                    fitness -= 0.4
            elif time_gap == 2:
                fitness += 0.25
            elif time_gap == 0:
                fitness -= 0.25

    return fitness


In [86]:
#@title Find Probability Distribution
from scipy.special import softmax
import random

def evaluate_population(population, activities, rooms):
    fitness_scores = []
    for chromosome in population:
        #print("Calling decode_chromosome() within evaluate_population")
        schedule = decode_chromosome(chromosome, activities, room_mapping, time_slot_mapping, facilitator_mapping)
        fitness = fitness_function(schedule, activities, rooms)
        fitness_scores.append((chromosome, fitness))

    fitness_values = [fitness for _, fitness in fitness_scores]
    probabilities = softmax(fitness_values)
    population_with_probs = [(chromosome, prob) for (chromosome, _), prob in zip(fitness_scores, probabilities)]
    return population_with_probs

In [139]:
#@title Selection, Crossover, and Mutation functions

def select_parents_softmax(population_with_probs):
    """Select two parents based on their softmax probabilities."""
    selected = []
    for _ in range(2):  # Select two parents
        r = random.uniform(0, 1)
        cumulative_prob = 0
        for chromosome, prob in population_with_probs:
            cumulative_prob += prob
            if cumulative_prob >= r:
                selected.append(chromosome)
                break
    return selected

def crossover(parent1, parent2, room_bits, time_slot_bits, facilitator_bits):
    """Perform segment-wise crossover between two parents to produce an offspring."""
    segment_length = room_bits + time_slot_bits + facilitator_bits
    offspring = ""

    # Process each activity's segments independently
    for i in range(0, len(parent1), segment_length):
        # Extract each parent's room, time, and facilitator segments
        parent1_room = parent1[i:i + room_bits]
        parent1_time = parent1[i + room_bits:i + room_bits + time_slot_bits]
        parent1_facilitator = parent1[i + room_bits + time_slot_bits:i + segment_length]

        parent2_room = parent2[i:i + room_bits]
        parent2_time = parent2[i + room_bits:i + room_bits + time_slot_bits]
        parent2_facilitator = parent2[i + room_bits + time_slot_bits:i + segment_length]

        # Perform segment-wise crossover by randomly choosing each segment from one parent
        offspring_room = parent1_room if random.random() < 0.5 else parent2_room
        offspring_time = parent1_time if random.random() < 0.5 else parent2_time
        offspring_facilitator = parent1_facilitator if random.random() < 0.5 else parent2_facilitator

        # Concatenate the selected segments for this activity
        offspring += offspring_room + offspring_time + offspring_facilitator

    return offspring

def mutate(chromosome, mutation_rate, activities, room_mapping, time_slot_mapping, facilitator_mapping):
    """Mutate the chromosome by randomly altering segments with a given mutation rate."""
     #Makes no changes if our mutation rate is 0 (saves calculation costs)
    if mutation_rate <= 0:
      return chromosome

    mutated_chromosome = list(chromosome)  # Convert to list for mutability
    room_bits = math.ceil(math.log2(len(room_mapping)))
    time_slot_bits = math.ceil(math.log2(len(time_slot_mapping)))
    facilitator_bits = math.ceil(math.log2(len(facilitator_mapping)))
    segment_length = room_bits + time_slot_bits + facilitator_bits  # Total bits per activity

    #mutation_triggered = False

    for i in range(0, len(chromosome), segment_length):
        if random.random() < mutation_rate:
            #mutation_triggered = True
            part_to_mutate = random.choice(['room', 'time', 'facilitator'])

            if part_to_mutate == 'room':
                new_room = random.choice(list(room_mapping.values()))  # Choose a random room binary value
                mutated_chromosome[i:i + room_bits] = new_room
            elif part_to_mutate == 'time':
                new_time = random.choice(list(time_slot_mapping.values()))  # Choose a random time slot binary value
                mutated_chromosome[i + room_bits:i + room_bits + time_slot_bits] = new_time
            elif part_to_mutate == 'facilitator':
                new_facilitator = random.choice(list(facilitator_mapping.values()))  # Choose a random facilitator binary value
                mutated_chromosome[i + room_bits + time_slot_bits:i + segment_length] = new_facilitator

    #if mutation_triggered:
       #print("Mutation occurred despite zero mutation rate. Check mutation logic.")

    return ''.join(mutated_chromosome)

In [77]:
pop_size = 500
population = generate_initial_population(pop_size, activities, room_mapping, time_slot_mapping, facilitator_mapping)

"""
#Printing for Debugging
print("Generated Population:")
for i, chromosome in enumerate(population):
    decoded_schedule = decode_chromosome(chromosome, activities, room_mapping, time_slot_mapping, facilitator_mapping)
    print(f"Chromosome {i+1}:")
    print("  Encoded:", chromosome)
    print("  Decoded:", decoded_schedule)

"""

'\n#Printing for Debugging\nprint("Generated Population:")\nfor i, chromosome in enumerate(population):\n    decoded_schedule = decode_chromosome(chromosome, activities, room_mapping, time_slot_mapping, facilitator_mapping)\n    print(f"Chromosome {i+1}:")\n    print("  Encoded:", chromosome)\n    print("  Decoded:", decoded_schedule)\n\n'

In [142]:
def genetic_algorithm(pop_size, generations, mutation_rate, activities, rooms, room_mapping, time_slot_mapping, facilitator_mapping):
    population = generate_initial_population(pop_size, activities, room_mapping, time_slot_mapping, facilitator_mapping)
    previous_avg_fitness = 0

    for generation in range(generations):
        # Step 1: Evaluate fitness and convert to probabilities
        population_with_probs = evaluate_population(population, activities, rooms)

        # Calculate avg and best fitness for tracking
        #print("Calling decode_chromosome() within genetic_algorithms for the fit_values call")
        fitness_values = [fitness_function(decode_chromosome(chromosome, activities, room_mapping, time_slot_mapping, facilitator_mapping), activities, rooms)
                          for chromosome in population]
        avg_fitness = sum(fitness_values) / pop_size
        best_fitness = max(fitness_values)

        print(f"Generation {generation + 1}, Best Fitness: {best_fitness:.4f}, Avg Fitness: {avg_fitness:.4f}")

        # Stopping criteria based on generation count and improvement threshold
        if generation >= 100 and (avg_fitness - previous_avg_fitness) / (previous_avg_fitness + 1e-9) < 0.01:
            break
        previous_avg_fitness = avg_fitness

        # Step 2: Reproduce with softmax-based selection
        new_population = []
        while len(new_population) < pop_size:
            # Select two parents
            parent1, parent2 = select_parents_softmax(population_with_probs)
            offspring = crossover(parent1, parent2, room_bits, time_slot_bits, facilitator_bits)
            offspring = mutate(offspring, mutation_rate, activities, room_mapping, time_slot_mapping, facilitator_mapping)


            new_population.append(offspring)

        population = new_population  # Update for the next generation

    # Return best schedule from the final generation
    #print("Calling decode_chromosome() within genetic_algorithm for the best_schedule call")
    best_schedule_chromosome = max(population, key=lambda chromo: fitness_function(decode_chromosome(chromo, activities, room_mapping, time_slot_mapping, facilitator_mapping), activities, rooms))
    best_schedule = decode_chromosome(best_schedule_chromosome, activities, room_mapping, time_slot_mapping, facilitator_mapping)
    best_fitness = fitness_function(best_schedule, activities, rooms)

    return best_schedule, best_fitness

def output_best_schedule(best_schedule):
    print("Best Schedule:")
    for activity, assignment in best_schedule.items():
        print(f"{activity}: Room={assignment['room']}, Time={assignment['time']}, Facilitator={assignment['facilitator']}\n")

# Run the genetic algorithm
pop_size = 500
generations = 200
mutation_rate = 0.1
best_schedule, best_fitness = genetic_algorithm(pop_size, generations, mutation_rate, activities, rooms, room_mapping, time_slot_mapping, facilitator_mapping)
print(f"Best Schedule Fitness: {best_fitness}")
output_best_schedule(best_schedule)

Generation 1, Best Fitness: 4.5000, Avg Fitness: -1.0100
Generation 2, Best Fitness: 5.8000, Avg Fitness: 1.2110
Generation 3, Best Fitness: 6.0000, Avg Fitness: 2.5086
Generation 4, Best Fitness: 7.1000, Avg Fitness: 3.3764
Generation 5, Best Fitness: 7.1000, Avg Fitness: 3.9712
Generation 6, Best Fitness: 7.6000, Avg Fitness: 4.4116
Generation 7, Best Fitness: 7.9000, Avg Fitness: 4.8426
Generation 8, Best Fitness: 8.0000, Avg Fitness: 5.2014
Generation 9, Best Fitness: 8.6000, Avg Fitness: 5.6182
Generation 10, Best Fitness: 8.6000, Avg Fitness: 5.7684
Generation 11, Best Fitness: 8.7000, Avg Fitness: 5.9840
Generation 12, Best Fitness: 9.0000, Avg Fitness: 6.2398
Generation 13, Best Fitness: 9.0000, Avg Fitness: 6.4090
Generation 14, Best Fitness: 9.1000, Avg Fitness: 6.6036
Generation 15, Best Fitness: 9.1000, Avg Fitness: 6.7042
Generation 16, Best Fitness: 9.0000, Avg Fitness: 6.7636
Generation 17, Best Fitness: 9.0000, Avg Fitness: 6.8098
Generation 18, Best Fitness: 9.4000, Av